Imports

In [106]:
import pandas as pd
import numpy as np
from pyomo.environ import * 
from pyomo.opt import SolverFactory
import folium
import csv
import random  
import copy

Cargar archivos requerido por la instancia

In [107]:
clients_df = pd.read_csv("Proyecto_Caso_Base/clients.csv")
depots_df = pd.read_csv("Proyecto_Caso_Base/depots.csv")
vehicles_df = pd.read_csv("Proyecto_Caso_Base/vehicles.csv")


In [108]:
print(clients_df.head())
print(depots_df.head())
print(vehicles_df.head())

   ClientID  LocationID  Demand  Longitude  Latitude
0         1           2      13 -74.098938  4.597954
1         2           3      15 -74.075571  4.687821
2         3           4      12 -74.107085  4.709494
3         4           5      15 -74.097280  4.605029
4         5           6      20 -74.164641  4.648464
   LocationID  DepotID  Longitude  Latitude  Capacity
0           1        1 -74.153536  4.743359        90
  VehicleType  Capacity  Range
0     Gas Car       130    170
1     Gas Car       140    200
2     Gas Car       120    180
3     Gas Car       100     90
4     Gas Car        70    100


In [109]:
C=list(depots_df["DepotID"])
CAP=dict(zip(depots_df["DepotID"],depots_df["Capacity"]))

In [110]:
print("Conjunto C:", C)
print("Capacidades CAP_i:", CAP)

Conjunto C: [1]
Capacidades CAP_i: {1: 90}


In [111]:
K=list(clients_df["ClientID"])
DEM=dict(zip(clients_df["ClientID"],clients_df["Demand"])) 
COORDENADAS_CLIENTES=dict(zip(clients_df["ClientID"],zip(clients_df["Longitude"],clients_df["Latitude"])))

In [112]:
print("Conjunto K: clientes", K)
print("Capacidades Demanda:", DEM)
print("Coordenadas clientes:", COORDENADAS_CLIENTES)

Conjunto K: clientes [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Capacidades Demanda: {1: 13, 2: 15, 3: 12, 4: 15, 5: 20, 6: 17, 7: 17, 8: 20, 9: 20, 10: 15, 11: 17, 12: 12, 13: 21, 14: 15, 15: 17, 16: 10, 17: 25, 18: 12, 19: 11, 20: 15, 21: 14, 22: 18, 23: 15, 24: 11}
Coordenadas clientes: {1: (-74.09893796560621, 4.59795431125545), 2: (-74.07557103763986, 4.687820646838871), 3: (-74.10708524062704, 4.70949446000624), 4: (-74.09727965657427, 4.605029068682624), 5: (-74.16464148202755, 4.648463876533332), 6: (-74.12083799988112, 4.662137416953968), 7: (-74.02213076607309, 4.697499030379109), 8: (-74.17207549744595, 4.649416884236942), 9: (-74.15615257246444, 4.606310650273935), 10: (-74.09041145358674, 4.557379705282216), 11: (-74.17802255204528, 4.591594072172954), 12: (-74.1015410917749, 4.7564172406324055), 13: (-74.09690889182339, 4.646217006050524), 14: (-74.1219200708342, 4.725912125314368), 15: (-74.0942948461378, 4.604168478560718), 1

Distancia harvesiana

In [113]:
def dist_haversiana(lon1,lat1,lon2,lat2):
  R=6371.0 #radio tierra
  lon1,lat1,lon2,lat2=map(np.radians,[lon1,lat1,lon2,lat2])
  dlon=lon2-lon1
  dlat=lat2-lat1
  a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
  c = 2 * np.arcsin(np.sqrt(a))
  return R * c

In [114]:
DIST={}
for i,depot in depots_df.iterrows():
  for j,client in clients_df.iterrows():
    di=depot["DepotID"]
    ci=client["ClientID"]
    distance=dist_haversiana(depot["Longitude"],depot["Latitude"],
                             client["Longitude"],client["Latitude"]
                             )
    DIST[(di,ci)]=round(distance,2)

In [115]:
for key, val in list(DIST.items())[:5]:
  print("La distancia entre el centro de distribucion", key[0], "y el cliente", key[1], "es de", val)

La distancia entre el centro de distribucion 1.0 y el cliente 1.0 es de 17.26
La distancia entre el centro de distribucion 1.0 y el cliente 2.0 es de 10.62
La distancia entre el centro de distribucion 1.0 y el cliente 3.0 es de 6.38
La distancia entre el centro de distribucion 1.0 y el cliente 4.0 es de 16.6
La distancia entre el centro de distribucion 1.0 y el cliente 5.0 es de 10.62


In [116]:
V=list(vehicles_df.index)
Q = dict(zip(V, vehicles_df["Capacity"]))


R = dict(zip(V, vehicles_df["Range"]))

In [117]:
print("Vehículos:", V)
print("Capacidades:", Q)
print("Rangos:", R)

Vehículos: [0, 1, 2, 3, 4, 5, 6, 7]
Capacidades: {0: 130, 1: 140, 2: 120, 3: 100, 4: 70, 5: 55, 6: 110, 7: 114}
Rangos: {0: 170, 1: 200, 2: 180, 3: 90, 4: 100, 5: 170, 6: 150, 7: 140}


In [118]:
for v in V:
    print(f"vehiculo {v}: Tipo={vehicles_df['VehicleType'][v]}, Capacidad={Q[v]}, Rango={R[v]}")

vehiculo 0: Tipo=Gas Car, Capacidad=130, Rango=170
vehiculo 1: Tipo=Gas Car, Capacidad=140, Rango=200
vehiculo 2: Tipo=Gas Car, Capacidad=120, Rango=180
vehiculo 3: Tipo=Gas Car, Capacidad=100, Rango=90
vehiculo 4: Tipo=Gas Car, Capacidad=70, Rango=100
vehiculo 5: Tipo=Gas Car, Capacidad=55, Rango=170
vehiculo 6: Tipo=Gas Car, Capacidad=110, Rango=150
vehiculo 7: Tipo=Gas Car, Capacidad=114, Rango=140


Implementar metodo metaheurístico: Algorimo genético 

Parametros

In [119]:
pupulation_size = 50
max_generations = 100
crossover_rate = 0.8
mutation_rate = 0.2
elitism_count = 2

Inicialización de la población

La población es el conjunto de individuos que representan posibles soluciones. En este caso un individuo representa una forma distinta de asignar clientes a vehículos, definir el orden de visita de esos clientes por vehículo y respetar la capacidad del vehículo.

In [ ]:
def inicializar_poblacion(poblacion_size, clients, cap_vehiculos, dem_clients):
    poblacion = []
    for _ in range(poblacion_size):
        clientes_barajados = clients.copy()
        np.random.shuffle(clientes_barajados)

        individuo = []
        ruta_actual = []
        carga_actual = 0
        capacidad_idx = 0

        for cliente in clientes_barajados:
            demanda = dem_clients[cliente]
            if carga_actual + demanda <= cap_vehiculos[capacidad_idx]:
                ruta_actual.append(cliente)
                carga_actual += demanda
            else:
                individuo.append(ruta_actual)
                capacidad_idx += 1
                if capacidad_idx >= len(cap_vehiculos):
                    break
                ruta_actual = [cliente]
                carga_actual = demanda

        if ruta_actual:
            individuo.append(ruta_actual)

        poblacion.append(individuo)
    return poblacion

def fitness(individual, distancias, deposito_id, coords_clientes, cap_vehiculos, dem_clients):
    costo_total = 0
    penalizacion = 0

    for idx, ruta in enumerate(individual):
        if not ruta:
            continue

        carga = sum(dem_clients[c] for c in ruta)
        if carga > cap_vehiculos[min(idx, len(cap_vehiculos)-1)]:
            penalizacion += 1000 * (carga - cap_vehiculos[min(idx, len(cap_vehiculos)-1)])

        costo_total += distancias[(deposito_id, ruta[0])]
        for i in range(len(ruta) - 1):
            c1, c2 = ruta[i], ruta[i+1]
            lon1, lat1 = coords_clientes[c1]
            lon2, lat2 = coords_clientes[c2]
            costo_total += dist_haversiana(lon1, lat1, lon2, lat2)
        costo_total += distancias[(deposito_id, ruta[-1])]

    return costo_total + penalizacion

def seleccion_torneo(population, fitness_scores, k=2):
    selected = []
    for _ in range(len(population)):
        indices = random.sample(range(len(population)), k)
        mejor_idx = min(indices, key=lambda i: fitness_scores[i])
        selected.append(copy.deepcopy(population[mejor_idx]))
    return selected

def crossover_rutas(p1, p2):
    hijo1, hijo2 = [], []

    split1 = random.randint(0, len(p1)-1)
    split2 = random.randint(0, len(p2)-1)

    r1 = p1[:split1]
    r2 = p2[split2:]

    flat_r1 = set(sum(r1, []))
    flat_r2 = set(sum(r2, []))

    restantes1 = [c for ruta in p2 for c in ruta if c not in flat_r1]
    restantes2 = [c for ruta in p1 for c in ruta if c not in flat_r2]

    hijo1 = r1 + [restantes1]
    hijo2 = r2 + [restantes2]

    return hijo1, hijo2

def mutar_individuo(individual):
    rutas_validas = [ruta for ruta in individual if len(ruta) >= 1]
    if len(rutas_validas) < 2:
        return

    r1, r2 = random.sample(rutas_validas, 2)
    i1 = random.randint(0, len(r1) - 1)
    i2 = random.randint(0, len(r2) - 1)
    r1[i1], r2[i2] = r2[i2], r1[i1]



# K clientes 
# Q capacidades de los vehiculos
# DEM demanda de los clientes
cap_vehiculos_transform = [Q[v] for v in V] 
population = inicializar_poblacion(pupulation_size, K, cap_vehiculos_transform, DEM)
# print("K",K)
print("CAP",cap_vehiculos_transform)
print("DEM",DEM)
print("cap vehiculos",)
# print("Dist: ", DIST)
# print("Distancia cliente: " , COORDENADAS_CLIENTES)

# for i, individuo in enumerate(population):
#     clientes_visitados = sum(individuo, [])  # Flatten all rutas
#     if len(clientes_visitados) != len(set(clientes_visitados)):
#         print(f"Individuo {i} tiene clientes repetidos")
#     else:
#         print(f"Individuo {i} es válido ")

DIST_correctos_ids = {(int(i), int(j)): float(d) for (i, j), d in DIST.items()}

# Evaluar la población inicial
fitness_scores = [
    fitness(ind, DIST_correctos_ids, 1, COORDENADAS_CLIENTES, cap_vehiculos_transform, DEM)
    for ind in population
]

# print("Poblacion inicial: ", population)
# print("Fitness scores: ", fitness_scores)

for generation in range(max_generations):
    # Selección
    mating_pool = seleccion_torneo(population, fitness_scores)

    # Cruce
    offspring = []
    for _ in range((pupulation_size - elitism_count) // 2):
        p1, p2 = random.sample(mating_pool, 2)
        if random.random() < crossover_rate:
            h1, h2 = crossover_rutas(p1, p2)
        else:
            h1, h2 = p1.copy(), p2.copy()
        offspring.extend([h1, h2])

    # Mutación
    for individual in offspring:
        if random.random() < mutation_rate:
            mutar_individuo(individual)

    # Evaluar descendencia
    # Evaluar descendencia
    offspring_fitness = [
        fitness(ind, DIST_correctos_ids, 1, COORDENADAS_CLIENTES, cap_vehiculos_transform, DEM)
        for ind in offspring
    ]

    # Elitismo sobre población actual
    elite_indices = sorted(range(len(fitness_scores)), key=lambda i: fitness_scores[i])[:elitism_count]
    elites = [copy.deepcopy(population[i]) for i in elite_indices]
    elite_fitness = [fitness_scores[i] for i in elite_indices]

    # Nueva población y sus fitness
    population = elites + offspring[:pupulation_size - elitism_count]
    fitness_scores = elite_fitness + offspring_fitness[:pupulation_size - elitism_count]

    # (Opcional) Registrar mejor fitness por generación

CAP [130, 140, 120, 100, 70, 55, 110, 114]
DEM {1: 13, 2: 15, 3: 12, 4: 15, 5: 20, 6: 17, 7: 17, 8: 20, 9: 20, 10: 15, 11: 17, 12: 12, 13: 21, 14: 15, 15: 17, 16: 10, 17: 25, 18: 12, 19: 11, 20: 15, 21: 14, 22: 18, 23: 15, 24: 11}
cap vehiculos


In [121]:
# Encontrar el mejor individuo después de todas las generaciones
mejor_idx = fitness_scores.index(min(fitness_scores))
mejor_individuo = population[mejor_idx]
mejor_costo = fitness_scores[mejor_idx]

print("\n🔎 Mejor solución encontrada:")
print("Costo total:", round(mejor_costo, 2))
print("Rutas asignadas:")
for i, ruta in enumerate(mejor_individuo):
    print(f"  Vehículo {i+1}: {ruta}")


🔎 Mejor solución encontrada:
Costo total: 196.5
Rutas asignadas:
  Vehículo 1: [6, 15, 4, 1, 22, 13, 23, 12]
  Vehículo 2: [18, 19, 21, 7, 8, 9, 24, 2]
  Vehículo 3: [14, 10, 16, 5, 3, 17]


In [124]:
import folium

# Encontrar el mejor individuo
mejor_idx = fitness_scores.index(min(fitness_scores))
mejor_individuo = population[mejor_idx]

# Crear el mapa centrado en Bogotá
mapa = folium.Map(location=[4.60971, -74.08175], zoom_start=12)

# Marcar el depósito (usando solo uno para este caso)
for _, depot in depots_df.iterrows():
    folium.Marker(
        location=[depot['Latitude'], depot['Longitude']], 
        popup=f"CD {depot['DepotID']}", 
        icon=folium.Icon(color='blue')
    ).add_to(mapa)

# Marcar todos los clientes
for _, client in clients_df.iterrows():
    folium.Marker(
        location=[client['Latitude'], client['Longitude']], 
        popup=f"Cliente {client['ClientID']}",
        icon=folium.Icon(color='green')
    ).add_to(mapa)

# Dibujar rutas del mejor individuo
for ruta in mejor_individuo:
    if not ruta:
        continue
    ruta_coords = []

    # Punto de inicio (depósito)
    cd_lat = depots_df.iloc[0]['Latitude']
    cd_lon = depots_df.iloc[0]['Longitude']
    ruta_coords.append([cd_lat, cd_lon])

    # Coordenadas de clientes en la ruta
    for cliente_id in ruta:
        client_lat, client_lon = COORDENADAS_CLIENTES[cliente_id][1], COORDENADAS_CLIENTES[cliente_id][0]
        ruta_coords.append([client_lat, client_lon])

    # Retorno al depósito
    ruta_coords.append([cd_lat, cd_lon])

    folium.PolyLine(ruta_coords, color='red', weight=2.5, opacity=1).add_to(mapa)

# Guardar el resultado
mapa.save("mejor_ruta.html")


In [125]:
import folium

mapa = folium.Map(location=[4.6, -74.1], zoom_start=12)
colors = ["red", "blue", "green", "orange", "purple", "darkred", "cadetblue", "gray"]

for i, ruta in enumerate(mejor_individuo):
    coords = [[COORDENADAS_CLIENTES[c][1], COORDENADAS_CLIENTES[c][0]] for c in ruta]
    coords.insert(0, [depots_df.iloc[0]["Latitude"], depots_df.iloc[0]["Longitude"]])  # desde depósito
    coords.append([depots_df.iloc[0]["Latitude"], depots_df.iloc[0]["Longitude"]])     # regreso

    folium.PolyLine(coords, color=colors[i % len(colors)], weight=3).add_to(mapa)

mapa.save("mejor_rutaOtraAlternativa.html")


Evaluación

Selección

Cruce(recombinación)

Mutación

Reemplazo

Iteración